# Previsão Avançada de Mercado Imobiliário - IDCI-VIX

Este notebook demonstra uma aplicação avançada do sistema de previsão usando:

- Construção do Índice Dinâmico de Confiança Imobiliária (IDCI-VIX)
- Seleção automática de variáveis via causalidade de Granger
- Múltiplos modelos de previsão (ARIMA, SARIMA, Markov-Switching, ML)
- Ensemble learning otimizado
- Validação temporal com cross-validation
- Análise de performance e diagnósticos completos

In [ ]:
import sys
import os

# Adicionar o diretório src ao path
sys.path.insert(0, os.path.abspath('../../src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Geração de Dados Sintéticos Realistas

Vamos criar um dataset sintético que simula variáveis macroeconômicas reais:

In [ ]:
def gerar_dados_realistas(n_periodos=120, seed=42):
    """
    Gera dados sintéticos realistas simulando variáveis macroeconômicas.
    
    Variáveis incluídas:
    - PIB: Tendência crescente com ciclos
    - Taxa Selic: Política monetária com mudanças de regime
    - IPCA: Inflação com persistência
    - Desemprego: Contra-cíclico ao PIB
    - Crédito Imobiliário: Relacionado ao PIB e Selic
    - Vendas Varejo: Indicador de consumo
    - Confiança do Consumidor: Sentimento de mercado
    """
    np.random.seed(seed)
    
    # Datas mensais
    datas = pd.date_range(start='2015-01-01', periods=n_periodos, freq='M')
    
    # Tendência temporal
    t = np.arange(n_periodos)
    
    # PIB - Crescimento com ciclos econômicos
    pib_tendencia = 2000 + 15 * t
    pib_ciclo = 200 * np.sin(2 * np.pi * t / 48) + 100 * np.sin(2 * np.pi * t / 24)
    pib = pib_tendencia + pib_ciclo + np.random.normal(0, 50, n_periodos)
    
    # Selic - Política monetária reativa à inflação
    selic_base = 10.0
    selic = np.zeros(n_periodos)
    selic[0] = selic_base
    for i in range(1, n_periodos):
        shock = np.random.normal(0, 0.3)
        # Mudanças de regime ocasionais
        if i % 30 == 0:
            shock += np.random.choice([-2, 2])
        selic[i] = np.clip(selic[i-1] + shock, 2.0, 20.0)
    
    # IPCA - Inflação com persistência
    ipca = np.zeros(n_periodos)
    ipca[0] = 0.5
    for i in range(1, n_periodos):
        # AR(1) com média móvel
        ipca[i] = 0.6 * ipca[i-1] + 0.3 + np.random.normal(0, 0.2)
        ipca[i] = np.clip(ipca[i], -1.0, 2.5)
    
    # Desemprego - Contra-cíclico
    desemprego_base = 10.0
    pib_normalizado = (pib - pib.mean()) / pib.std()
    desemprego = desemprego_base - 2 * pib_normalizado + np.random.normal(0, 0.5, n_periodos)
    desemprego = np.clip(desemprego, 4.0, 16.0)
    
    # Crédito Imobiliário - Positivo com PIB, negativo com Selic
    credito_tendencia = 50000 + 400 * t
    credito_ciclo = 5000 * pib_normalizado - 2000 * (selic - selic.mean()) / selic.std()
    credito = credito_tendencia + credito_ciclo + np.random.normal(0, 2000, n_periodos)
    
    # Vendas Varejo - Relacionado ao PIB e confiança
    vendas_base = 100
    vendas = vendas_base + 10 * np.sin(2 * np.pi * t / 12) + 5 * pib_normalizado
    vendas += np.random.normal(0, 3, n_periodos)
    
    # Confiança do Consumidor - Sentimento geral
    confianca = 100 + 15 * pib_normalizado - 10 * (desemprego - desemprego.mean()) / desemprego.std()
    confianca += np.random.normal(0, 5, n_periodos)
    
    # Criar DataFrame
    df = pd.DataFrame({
        'data': datas,
        'pib_real': pib,
        'taxa_selic': selic,
        'ipca': ipca,
        'taxa_desemprego': desemprego,
        'credito_imobiliario': credito,
        'vendas_varejo': vendas,
        'confianca_consumidor': confianca
    })
    
    df.set_index('data', inplace=True)
    
    return df

# Gerar dados
df = gerar_dados_realistas(n_periodos=120)
print("Dataset gerado:")
print(f"Período: {df.index[0].strftime('%Y-%m')} a {df.index[-1].strftime('%Y-%m')}")
print(f"Observações: {len(df)}")
print(f"\nPrimeiras linhas:")
df.head()

## 2. Análise Exploratória dos Dados

In [ ]:
# Estatísticas descritivas
print("Estatísticas Descritivas:")
print("=" * 80)
df.describe()

In [ ]:
# Visualizar séries temporais
fig, axes = plt.subplots(4, 2, figsize=(16, 14))
fig.suptitle('Séries Temporais das Variáveis Macroeconômicas', fontsize=16, fontweight='bold')

for idx, col in enumerate(df.columns):
    ax = axes[idx // 2, idx % 2]
    ax.plot(df.index, df[col], linewidth=2)
    ax.set_title(col.replace('_', ' ').title(), fontsize=12, fontweight='bold')
    ax.set_xlabel('Data', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlação
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação das Variáveis', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 3. Testes de Estacionaridade e Transformações

Aplicar testes ADF e KPSS para determinar se as séries precisam de diferenciação:

In [ ]:
from preprocessing.stationarity import StationarityTester# Testar estacionaridadetester = StationarityTester(alpha=0.05)print("Testes de Estacionaridade (ADF e KPSS)")print("=" * 80)resultados_estacionaridade = {}for col in df.columns:    result = tester.test_stationarity(df[col])        is_stationary = result['is_stationary']    adf_pval = result['adf']['pvalue']    kpss_pval = result['kpss']['pvalue']        resultados_estacionaridade[col] = {        'estacionaria': is_stationary,        'adf_pvalue': adf_pval,        'kpss_pvalue': kpss_pval    }        status = "✓ Estacionária" if is_stationary else "✗ Não estacionária"    print(f"{col:25s} {status:20s} | ADF p={adf_pval:.4f} | KPSS p={kpss_pval:.4f}")print("" + "=" * 80)

In [ ]:
# Aplicar transformações para tornar séries estacionárias
df_transformed = df.copy()
transformacoes = {}

for col in df.columns:
    if not resultados_estacionaridade[col]['estacionaria']:
        # Aplicar primeira diferença
        df_transformed[col] = df[col].diff()
        transformacoes[col] = 'diff(1)'
    else:
        transformacoes[col] = 'none'

# Remover NaNs gerados pela diferenciação
df_transformed = df_transformed.dropna()

print("Transformações aplicadas:")
for col, trans in transformacoes.items():
    print(f"  {col:25s} -> {trans}")

## 4. Seleção de Variáveis via Causalidade de Granger

Identificar quais variáveis têm poder preditivo para o PIB:

In [ ]:
from preprocessing.variable_selection import GrangerSelector

# Selecionar variáveis que causam Granger no PIB
target = 'pib_real'
selector = GrangerSelector(max_lag=6, alpha=0.05)

selected_vars = selector.select_variables(
    df_transformed,
    target_col=target
)

print(f"\nVariáveis selecionadas (Granger-causam '{target}'):")
print("=" * 80)
for var in selected_vars:
    if var != target:
        print(f"  ✓ {var}")
print("\n" + "=" * 80)

## 5. Construção do Índice IDCI-VIX via Modelo de Fatores Dinâmicos

Usar Filtro de Kalman para extrair um fator latente comum:

In [ ]:
from factor_model.dynamic_factor import DynamicFactorModel

# Preparar dados para o modelo de fatores
X_factor = df_transformed[selected_vars].values

# Estimar modelo de fator dinâmico
dfm = DynamicFactorModel(n_factors=1)
dfm.fit(X_factor)

# Extrair fator (IDCI-VIX bruto)
factor_raw = dfm.factors_.flatten()

# Normalizar para escala 0-10
factor_min = factor_raw.min()
factor_max = factor_raw.max()
idci_vix = 10 * (factor_raw - factor_min) / (factor_max - factor_min)

# Adicionar ao DataFrame
df_transformed['IDCI_VIX'] = idci_vix

print(f"Índice IDCI-VIX construído:")
print(f"  Média: {idci_vix.mean():.2f}")
print(f"  Desvio: {idci_vix.std():.2f}")
print(f"  Mín/Máx: {idci_vix.min():.2f} / {idci_vix.max():.2f}")

In [ ]:
# Visualizar IDCI-VIX
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df_transformed.index, idci_vix, linewidth=2.5, color='darkblue', label='IDCI-VIX')
ax.axhline(y=5, color='red', linestyle='--', alpha=0.5, label='Mediana (5.0)')
ax.fill_between(df_transformed.index, 0, 10, alpha=0.1, color='blue')
ax.set_title('Índice Dinâmico de Confiança Imobiliária - Vitória (IDCI-VIX)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Índice (0-10)', fontsize=12)
ax.set_ylim(0, 10)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Modelagem e Previsão com Múltiplos Modelos

Treinar vários modelos e comparar performance:

In [ ]:
from forecasting.arima import ARIMAForecaster
from forecasting.sarima import SARIMAForecaster
from forecasting.markov_switching import MarkovSwitchingForecaster
from forecasting.ridge import RidgeForecaster
from forecasting.random_forest import RandomForestForecaster

# Dividir dados: 80% treino, 20% teste
train_size = int(0.8 * len(df_transformed))
train_data = df_transformed.iloc[:train_size]
test_data = df_transformed.iloc[train_size:]

target_col = 'IDCI_VIX'
y_train = train_data[target_col].values
y_test = test_data[target_col].values
n_forecast = len(y_test)

print(f"Tamanho do conjunto de treino: {len(train_data)}")
print(f"Tamanho do conjunto de teste: {len(test_data)}")
print(f"Horizonte de previsão: {n_forecast} períodos")

In [ ]:
# Dicionário para armazenar previsões
previsoes = {}

# 1. ARIMA
print("Treinando ARIMA...")
arima = ARIMAForecaster(order=(2, 0, 2))
arima.fit(y_train)
previsoes['ARIMA'] = arima.forecast(n_forecast)

# 2. SARIMA
print("Treinando SARIMA...")
sarima = SARIMAForecaster(order=(1, 0, 1), seasonal_order=(1, 0, 1, 12))
sarima.fit(y_train)
previsoes['SARIMA'] = sarima.forecast(n_forecast)

# 3. Markov-Switching
print("Treinando Markov-Switching...")
ms = MarkovSwitchingForecaster(n_regimes=2, order=2)
ms.fit(y_train)
previsoes['Markov-Switching'] = ms.forecast(n_forecast)

# 4. Ridge Regression
print("Treinando Ridge...")
X_train = train_data[selected_vars].values
X_test = test_data[selected_vars].values
ridge = RidgeForecaster(alpha=1.0, lags=3)
ridge.fit(X_train, y_train)
previsoes['Ridge'] = ridge.forecast(X_test)

# 5. Random Forest
print("Treinando Random Forest...")
rf = RandomForestForecaster(n_estimators=100, max_depth=10, lags=5)
rf.fit(X_train, y_train)
previsoes['Random Forest'] = rf.forecast(X_test)

print("\n✓ Todos os modelos treinados com sucesso!")

## 7. Avaliação e Comparação de Modelos

In [ ]:
from evaluation.metrics import calculate_metrics

# Calcular métricas para cada modelo
metricas_modelos = {}

for nome_modelo, y_pred in previsoes.items():
    metrics = calculate_metrics(y_test, y_pred)
    metricas_modelos[nome_modelo] = metrics

# Criar DataFrame com métricas
df_metricas = pd.DataFrame(metricas_modelos).T

print("\nPerformance dos Modelos:")
print("=" * 80)
print(df_metricas.round(4))
print("\n" + "=" * 80)

# Identificar melhor modelo por métrica
print("\nMelhores Modelos por Métrica:")
for metrica in df_metricas.columns:
    if metrica in ['mae', 'rmse', 'mape']:
        melhor = df_metricas[metrica].idxmin()
        valor = df_metricas.loc[melhor, metrica]
    else:
        melhor = df_metricas[metrica].idxmax()
        valor = df_metricas.loc[melhor, metrica]
    print(f"  {metrica.upper():10s}: {melhor:20s} ({valor:.4f})")

In [ ]:
# Visualizar previsões vs valores reais
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
fig.suptitle('Comparação: Previsões vs Valores Reais', fontsize=16, fontweight='bold')

for idx, (nome_modelo, y_pred) in enumerate(previsoes.items()):
    ax = axes[idx // 2, idx % 2]
    
    # Plot
    ax.plot(test_data.index, y_test, 'o-', label='Real', linewidth=2, markersize=4)
    ax.plot(test_data.index, y_pred, 's--', label='Previsto', linewidth=2, markersize=4, alpha=0.7)
    
    # Adicionar métricas no título
    rmse = metricas_modelos[nome_modelo]['rmse']
    mape = metricas_modelos[nome_modelo]['mape']
    ax.set_title(f"{nome_modelo}\nRMSE: {rmse:.3f} | MAPE: {mape:.2f}%", 
                 fontsize=11, fontweight='bold')
    
    ax.set_xlabel('Data')
    ax.set_ylabel('IDCI-VIX')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# Remover subplot extra
fig.delaxes(axes[2, 1])

plt.tight_layout()
plt.show()

## 8. Ensemble Learning - Combinação de Modelos

In [ ]:
from evaluation.ensemble import EnsembleCombiner

# Preparar previsões como matriz
predictions_matrix = np.column_stack([previsoes[m] for m in previsoes.keys()])

# Criar ensemble com diferentes estratégias
ensemble_strategies = ['simple_average', 'weighted_average', 'median']
ensemble_results = {}

for strategy in ensemble_strategies:
    combiner = EnsembleCombiner(method=strategy)
    combiner.fit(predictions_matrix, y_test)
    y_ensemble = combiner.predict(predictions_matrix)
    
    metrics = calculate_metrics(y_test, y_ensemble)
    ensemble_results[f"Ensemble ({strategy})"] = {
        'predictions': y_ensemble,
        'metrics': metrics
    }

# Comparar ensemble com modelos individuais
print("\nPerformance do Ensemble vs Modelos Individuais:")
print("=" * 80)

# Adicionar métricas de ensemble ao DataFrame
for name, result in ensemble_results.items():
    df_metricas.loc[name] = result['metrics']

print(df_metricas.round(4))
print("\n" + "=" * 80)

# Melhor modelo geral
melhor_modelo_geral = df_metricas['rmse'].idxmin()
print(f"\n🏆 Melhor Modelo (menor RMSE): {melhor_modelo_geral}")
print(f"   RMSE: {df_metricas.loc[melhor_modelo_geral, 'rmse']:.4f}")

In [ ]:
# Visualizar comparação de ensemble
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(test_data.index, y_test, 'o-', label='Real', linewidth=3, markersize=6, color='black')

colors = ['blue', 'green', 'red']
for idx, (name, result) in enumerate(ensemble_results.items()):
    ax.plot(test_data.index, result['predictions'], 's--', 
            label=name, linewidth=2, markersize=5, alpha=0.7, color=colors[idx])

ax.set_title('Comparação de Estratégias de Ensemble', fontsize=14, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('IDCI-VIX', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Análise de Resíduos e Diagnósticos

In [ ]:
from scipy import stats

# Selecionar melhor modelo para análise de resíduos
modelo_analise = 'ARIMA'
residuos = y_test - previsoes[modelo_analise]

# Criar figura com múltiplos diagnósticos
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Resíduos ao longo do tempo
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(test_data.index, residuos, 'o-', linewidth=2)
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.fill_between(test_data.index, -2*residuos.std(), 2*residuos.std(), alpha=0.2)
ax1.set_title(f'Resíduos do Modelo {modelo_analise}', fontsize=12, fontweight='bold')
ax1.set_ylabel('Resíduo')
ax1.grid(True, alpha=0.3)

# 2. Histograma dos resíduos
ax2 = fig.add_subplot(gs[1, 0])
ax2.hist(residuos, bins=15, edgecolor='black', alpha=0.7)
ax2.set_title('Distribuição dos Resíduos', fontsize=11, fontweight='bold')
ax2.set_xlabel('Resíduo')
ax2.set_ylabel('Frequência')
ax2.grid(True, alpha=0.3, axis='y')

# 3. Q-Q Plot
ax3 = fig.add_subplot(gs[1, 1])
stats.probplot(residuos, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normalidade)', fontsize=11, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. ACF dos resíduos
from statsmodels.graphics.tsaplots import plot_acf
ax4 = fig.add_subplot(gs[2, 0])
plot_acf(residuos, lags=min(20, len(residuos)//2), ax=ax4)
ax4.set_title('Autocorrelação dos Resíduos', fontsize=11, fontweight='bold')

# 5. Scatter: Previsto vs Real
ax5 = fig.add_subplot(gs[2, 1])
ax5.scatter(y_test, previsoes[modelo_analise], alpha=0.6, s=50)
ax5.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='Linha ideal')
ax5.set_xlabel('Valores Reais')
ax5.set_ylabel('Valores Previstos')
ax5.set_title('Previsto vs Real', fontsize=11, fontweight='bold')
ax5.legend()
ax5.grid(True, alpha=0.3)

plt.suptitle(f'Diagnósticos Completos - {modelo_analise}', 
             fontsize=14, fontweight='bold', y=0.995)
plt.show()

# Testes estatísticos
print("\nTestes Estatísticos dos Resíduos:")
print("=" * 60)

# Teste de normalidade (Shapiro-Wilk)
stat_sw, p_sw = stats.shapiro(residuos)
print(f"Shapiro-Wilk (Normalidade): p-value = {p_sw:.4f}")
if p_sw > 0.05:
    print("  ✓ Resíduos parecem seguir distribuição normal")
else:
    print("  ✗ Resíduos não seguem distribuição normal")

# Teste de média zero
stat_t, p_t = stats.ttest_1samp(residuos, 0)
print(f"\nTeste t (Média = 0): p-value = {p_t:.4f}")
if p_t > 0.05:
    print("  ✓ Resíduos têm média não significativamente diferente de zero")
else:
    print("  ✗ Resíduos têm viés sistemático")

print("\n" + "=" * 60)

## 10. Conclusões e Recomendações

### Principais Achados:

1. **Construção do IDCI-VIX**: O índice foi construído com sucesso usando modelo de fatores dinâmicos
2. **Seleção de Variáveis**: Granger causality identificou variáveis relevantes
3. **Performance dos Modelos**: Comparamos 5 modelos diferentes com métricas rigorosas
4. **Ensemble**: A combinação de modelos geralmente melhora a performance

### Próximos Passos:

- Validação cruzada temporal mais robusta
- Otimização de hiperparâmetros via grid search
- Análise de cenários (pessimista/base/otimista)
- Implementação em produção com monitoramento contínuo